## TDC Recife 2020

### Setup

Os passos de setup abaixo assumem que o ambiente do Oracle Data Science foi provisionado usando o template oficial do Oracle Resource Manager (ORM). (Atual em Out/2020)

#### Rodar o notebook "Getting Started" para configurar acesso para `resource_principal` ou `api_key`

O notebook Getting Started é uma mistura de configuração inicial e demo. A parte de configuração é necessária antes de executar qualquer outro notebook. Você deve escolher o tipo de autenticação entre `resource principal` ou `api_key`.

Para usar a autenticação `resource_principal`, além dos passos contidos no notebook você vai precisar fazer os seguintes passos adicionais:

- O ORM por padrao não configura o DynamicGroup e as Policies para acessar recursos externos como o ObjectStorage.

Adicionar em Identity > Dynamic Groups > Dynamic Group Details a `Matching Rule` abaixo:

```
ALL {resource.type = 'datasciencenotebooksession'}
```

Adicionar em Identity > Policies > DataSciencePolicies o `Statement` abaixo:

```
allow dynamic-group DataScienceDynamicGroup to manage objects in tenancy
```

Esta regra adiciona o recurso de leitura de arquivos do Object Storage (schemas `oci://` e `ocis://`).

Para outras regras, ver a [documentação](https://docs.cloud.oracle.com/en-us/iaas/data-science/using/configure-tenancy.htm#creating_dynamic_groups_and_policies) oficial.

### Instalar o pyspark localmente

Rodar `pip install pyspark` numa janela de terminal ou numa célula bash (`%%bash`)


### Interagindo com a OCI

Importar o pacote ads para interagir com a Oracle Cloud e autenticar o usuário.

In [ ]:
import ads
from ads.dataset.factory import DatasetFactory

ads.set_auth(auth='resource_principal')

Carregar o arquivo TXT do Oracle Cloud Object Storage. 

Nota: Modifique o exemplo para usar o seu próprio bucket. O arquivo original pode ser baixado [aqui]("https://ocw.mit.edu/ans7870/6/6.006/s08/lecturenotes/files/t8.shakespeare.txt").

In [ ]:
ds = DatasetFactory.open("oci://danicat/shakespeare.txt", format="csv", delimiter="\n", header=None)

### Carregar arquivo texto para a memória do driver

In [ ]:
text = ds.compute()["0"].values.tolist()

### Inicializar Spark

In [ ]:
import pyspark
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.appName("TDC Recife").getOrCreate()

In [ ]:
spark

In [ ]:
sc = spark.sparkContext

In [ ]:
sc

### Word Count

In [ ]:
rdd = sc.parallelize(text)

In [ ]:
rdd

In [ ]:
rdd.take(10)

In [ ]:
words = rdd.flatMap(lambda line: line.lower().split())

In [ ]:
words.take(10)

In [ ]:
ones = words.map(lambda w: (w, 1))

In [ ]:
ones.take(10)

In [ ]:
counts = ones.reduceByKey(lambda x, y: x + y)

In [ ]:
counts.take(10)

In [ ]:
print(counts.toDebugString().decode('utf-8'))

In [ ]:
local = counts.collect()

In [ ]:
df = counts.toDF(["word", "count"])

In [ ]:
df.show()

In [ ]:
df.orderBy(df["count"].desc())

In [ ]:
df.orderBy(df["count"].desc()).show()

In [ ]:
df.createOrReplaceTempView("wordcount")

In [ ]:
spark.sql("select * from wordcount order by 2 desc").show()

In [ ]:
df.printSchema()

In [ ]:
df.explain()

### Fazendo o join the dois datasets

In [ ]:
import nltk
from nltk.corpus import stopwords

# nota: na primeira vez que rodar a nltk vai pedir para fazer o download do dicionario

In [ ]:
sw = stopwords.words('english')

In [ ]:
sw[0:10]

In [ ]:
swdf = spark.createDataFrame(sw, ["word"]) # nao vai funcionar!

In [ ]:
swdf = spark.createDataFrame(map(lambda x: (x,), sw), ["word"])

In [ ]:
swdf.show()

In [ ]:
swdf.createOrReplaceTempView("stopwords")

In [ ]:
spark.sql("select * from stopwords").show()

In [ ]:
clean_df = spark.sql("select word, count from wordcount w where not exists (select 1 from stopwords s where s.word = w.word) order by 2 desc")

In [ ]:
clean_df.show()

In [ ]:
from pyspark.sql.functions import broadcast

In [ ]:
bcsw = broadcast(swdf)

In [ ]:
bcsw.explain(True)

In [ ]:
bcsw.createOrReplaceTempView("bc_stopwords")

In [ ]:
bc_clean_df = spark.sql("select w.word, w.count from wordcount w where not exists (select 1 from bc_stopwords s where s.word = w.word) order by 2 desc")

In [ ]:
bc_clean_df.show()

In [ ]:
bc_clean_df.explain()

In [ ]:
clean_df.explain()

In [ ]:
bc_clean_df.explain(True)